# **Linear-nonlinear-Poisson model – LNPM**

The LNP model is a simplified model that has proven useful to describe the output of single neurons. The steps for  implementing it are described in the following schematic:  

<img src="LNP_flowchart.png">


This diagram also serves to outline the steps you'll need to take to make a simulation with the LNP model.

1). The stimulus. This one's easy. If you remember, the contrast values or the sequence of patterns that make up the stimuli we use are randomized by drawing random numbers. For the simulation, a sequence of random numbers suffices as stimulus. 

2). Under this model, a linear filter will determine (partly) the neuron's spiking output by filtering the signal. In very general terms, filtering means letting through what we want while letting out what we don't. A neuron wants signals to go through that are as similar as possible to its filter, while ignoring signals that differ from it. The neuron has no business dealing with those. In the simulation, to do the filtering, we should somehow compare the filter with the entire signal. The mathematical operation for this is a convolution, which will "slide" the filter along the signal making a point-to-point multiplication at each step. Similarities will produce larger positive values, whereas discrepancies will produce smaller or negative values. 

<img src="convolution.gif">


3). The result of convolving ("filtering") the signal with a linear filter is called the generator.

4). Once you have the generator, you will apply a **nonlinearity** to it in order to define the firing rate at each point.

5). One of the assumptions of the LNP model is that spiking is a Poisson process: spikes are stochastic and independent from each other, and the average rate (spikes per interval) is constant, although the probability of having a spike at any given time point is zero. For you to decide whether a given firing rate produces a spike or not, you multiply it by a time bin of your choice (the same for all) and compare it to a random number. If the random numbers is smaller, there's a spike; if it's not, there's no spike.

6). Now that you have the spikes, you should to be able to recover the filter (the STA) as you did earlier with the real cell data.